<a href="https://colab.research.google.com/github/HansHenseler/DFRWS-APAC-LLM-Workshop/blob/main/Part_III_Hands_on_with_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part III: Hands on with Llama2

This is an example for Instruction Answering.
You give an instruction to the LLM and the LLM makes the responses


Model used: llama-2-13b-chat.Q4_0.gguf
Model: Llama 2,
Size: 13B,
Model type: Chat model
Quantization: 4bit, native method
Model Format: gguf

First step: GET A GPU !!!


## Stage 1: Then install langchain and llama-cpp-python

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.0 MB/s eta 0:00:00


Install Llamma cpp python

In [2]:
!export LLAMA_CUBLAS=1
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.11-cp310-cp310-manylinux_2_35_x86_64.whl size=6423392 sha256=418af8aaffd8958a928e348d47ad2f5630f202e597a2403f6f74902e6c2f8a61
  Stored in directory: /root/.cache/pip/wheels/dc/42/77/a3ab0d02700427ea364de5797786c0272779dce795f62c3bc2
Successfully built llama-cpp-python


## Stage 2: Get the chat model

In [3]:
# This is the chat model!
!wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf

--2023-10-16 02:53:13--  https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.55, 18.164.174.17, 18.164.174.23, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/8d/b1/8db1d1f73b4caa58e947ccbfe2fb27ac5e495c2ad8457ad299d15987aee3b520/eda2a15d532bea4ce6fc14d15c2b72638243396816252f73a94dceeb0429112f?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-13b-chat.Q4_0.gguf%3B+filename%3D%22llama-2-13b-chat.Q4_0.gguf%22%3B&Expires=1697674874&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzY3NDg3NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84ZC9iMS84ZGIxZDFmNzNiNGNhYTU4ZTk0N2NjYmZlMmZiMjdhYzVlNDk1YzJhZDg0NTdhZDI5OWQxNTk4N2FlZTNiNTIwL2VkYTJhMTVkNTMyYmVhNGNlNmZjMTRkMTVjMmI3MjYzODI0Mz

In [5]:
from langchain.llms import LlamaCpp
# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/content/llama-2-13b-chat.Q4_0.gguf", # location of the model
    temperature=0.75,                 # temperature
    max_tokens=2000,                 # Max. number of tokens to be generated
    top_p=0.9,                    # top_p = 0.9
    top_k=30,                     # top_k = 30
    n_gpu_layers=43,                 # number of layers to offload to GPU
    verbose=True, # Verbose is required to pass to the callback manager
    n_batch=200,          # number of token generation in parallel
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [2]:
!nvidia-smi

Mon Oct 16 02:56:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Stage 3: try the model

In [ ]:
llm("what is water?")

"\n\nWater (H2O) is a clear, colorless, odorless liquid that is essential for life on Earth. It is the most abundant substance on our planet, covering about 71% of its surface and making up approximately 60% of the human body. Water is a vital component in many biological processes, including photosynthesis, metabolism, and reproduction.\n\nThe chemical composition of water is simple: it consists of two hydrogen atoms and one oxygen atom. The molecule is polar, meaning that the oxygen atom has a slightly negative charge and the hydrogen atoms have a slightly positive charge. This polarity gives water many of its unique properties and allows it to play a crucial role in various biological and chemical processes.\n\nSome of the key properties of water include:\n\n1. High specific heat capacity: Water has a high specific heat capacity, meaning that it can absorb and release a lot of heat energy without a large change in temperature. This property helps regulate the Earth's climate and wea

In [ ]:
llm("plan a 2 days trip in Singapore")

Llama.generate: prefix-match hit


"\n\nI'm planning to visit Singapore for 2 days, I would like some suggestions on what I can do and see during this time. I would be arriving at Changi airport in the morning and leaving in the evening.\n\nI would appreciate any recommendations on the following:\n\n* Accommodation (preferably within walking distance from a MRT station)\n* Must-see attractions\n* Food places to try\n* Public transportation options\n* Safety tips\n\nPlease let me know if you have any specific questions or requirements for your recommendations. Thank you!"

In [ ]:
txt = '''
1. Gardens by the Bay - I want to see the Supertree Grove and the Flower Dome and Cloud Forest Domes.
2. Marina Bay Sands - I want to take a photo with the iconic hotel and enjoy the light and water show at night.
3. Merlion - I want to see the famous statue and take a photo with it.
4. Singapore Flyer - I want to ride the giant Ferris wheel and enjoy the panoramic view of the city.
5. Chinatown - I would like to explore the colorful streets and try some delicious street food.
6. Little India - I would like to experience the vibrant Indian culture and try some authentic Indian cuisine.
7. Haw Par Villa - I want to see the largest theme park in Singapore and enjoy the Chinese mythology-themed rides and attractions.
8. Clarke Quay - I would like to visit the nightlife district and enjoy the bars, clubs, and live music performances.
'''
llm("Please summarize the following passage in 50 words. Passage: " + txt)

Llama.generate: prefix-match hit


'\nHere is a summary of the passage in 50 words:\n\nI want to visit Gardens by the Bay, Marina Bay Sands, Merlion, Singapore Flyer, Chinatown, Little India, Haw Par Villa, and Clarke Quay to experience the vibrant culture and attractions that Singapore has to offer.'

In [ ]:
llm("which countries are located in Europe")

ValueError: ignored

## Stage 4: Create a conversational agent


In [1]:
from langchain.llms import LlamaCpp
model_path = "/content/llama-2-13b-chat.Q4_0.gguf"
llm = LlamaCpp(model_path=model_path,
               # n_ctx=4096,
               seed=0,
               temperature=0.1,
               top_k=30,  # original 50
               top_p=0.75,# original 0.95
               n_batch=300,      # num. of parallel generation
               n_gpu_layers=43,  # important for GPU
               n_threads = 10,   # important for CPU
               max_tokens=1024,
               repeat_penalty=1.18,
               )

history = []

def convert_message_and_history_to_llama2_chat_format(message: str, memory_limit = 7) -> str:
    # memory_limit set to 7 -> only memorize latest 7 pairs of "user, assistant" conversation
    global history

    SYSTEM_PROMPT = """<s>[INST] <<SYS>>
    You are a helpful bot. Your answers are clear and concise.
    <</SYS>>

    """

    if len(history) > memory_limit:
        history = history[-memory_limit:]

    if len(history) == 0:
        return SYSTEM_PROMPT + f"{message} [/INST]"

    # first message pair is treated differently
    llama2_history_formated_message = SYSTEM_PROMPT + f"user{history[0][0]} [/INST] {history[0][1]} </s>"
    # for 2nd and further message pairs
    for user_msg, assistant_msg in history[1:]:
        llama2_history_formated_message += f"<s>[INST] {user_msg} [/INST] {assistant_msg} </s>"
    # for the current user message
    llama2_history_formated_message += f"<s>[INST] {message} [/INST]"

    return llama2_history_formated_message


def generate_response(message: str) -> str:
    global history

    query = convert_message_and_history_to_llama2_chat_format(message)
    responses = llm(query)
    history.append((message, responses))
    print("user: ", message)
    print("assistant: ", responses)
    print('\n')


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [2]:
generate_response("hi")
generate_response("what is iron?")
generate_response("what is the boiling point of it?")

user:  hi
assistant:    Hello! I'm here to help answer any questions you may have. What can I assist you with today?




Llama.generate: prefix-match hit


user:  what is iron?
assistant:    Iron is a chemical element with the symbol Fe and atomic number 26. It is a metal that belongs to the group of transition metals and is one of the most common elements on Earth.

Iron is an essential nutrient for many living organisms, including humans, where it plays a crucial role in the functioning of red blood cells and the transportation of oxygen throughout the body. It is also a key component in the synthesis of hemoglobin, which is the protein in red blood cells that carries oxygen.

In addition to its biological importance, iron is also a critical material in many industrial processes, such as the production of steel, which is an alloy of iron and carbon. Steel is a key component in construction, transportation, and other industries, and it is estimated that over 90% of all metal produced worldwide is steel.

Iron is also a significant component in the Earth's core, where it is thought to make up about 85% of the planet's total iron content. 

Llama.generate: prefix-match hit


user:  what is the boiling point of it?
assistant:    The boiling point of iron is approximately 2860°C (5176°F) at standard atmospheric pressure. This means that iron melts at a temperature of around 1538°C (2800°F) and boils at a temperature of around 2860°C (5176°F). However, it's important to note that the exact boiling point of iron can vary depending on the specific conditions in which it is being he


